<h1>Содержание<span class="tocSkip"></span></h1>
<br>
<div class="toc">
    <ul class="toc-item">
        <li>
            <span>
                <a href="#1-Подготовка-окружения">
                    <span class="toc-item-num">1&nbsp;&nbsp;</span>
                    Подготовка окружения
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#2-Загрузка-данных">
                    <span class="toc-item-num">2&nbsp;&nbsp;</span>
                    Загрузка данных
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#3-Трансформерная-архитектура-GPT">
                    <span class="toc-item-num">3&nbsp;&nbsp;</span>
                    Трансформерная архитектура GPT
                </a>
            </span>
            <ul class="toc-item">
                <li>
                    <span>
                        <a href="#3.1-Токенизация-символов">
                            <span class="toc-item-num">3.1&nbsp;&nbsp;</span>
                            Токенизация символов
                        </a>
                    </span>
                </li>
                <li>
                    <span>
                        <a href="#3.2-Подготовка-модели">
                            <span class="toc-item-num">3.2&nbsp;&nbsp;</span>
                            Подготовка модели
                        </a>
                    </span>
                </li>
                <li>
                    <span>
                        <a href="#3.3-Обучение-модели">
                            <span class="toc-item-num">3.3&nbsp;&nbsp;</span>
                            Обучение модели
                        </a>
                    </span>
                </li>
                <li>
                    <span>
                        <a href="#3.4-Генерация-текста">
                            <span class="toc-item-num">3.4&nbsp;&nbsp;</span>
                            Генерация текста
                        </a>
                    </span>
                </li>
            </ul>
        </li>
        <li>
            <span>
                <a href="#4-Общий-вывод">
                    <span class="toc-item-num">4&nbsp;&nbsp;</span>
                    Общий вывод
                </a>
            </span>
        </li>
    </ul>
</div>

# Генеративные текстовые нейросети | Архитектура GPT

**Постановка задачи:** натренировать и сравнить качество нескольких генеративных текстовых моделей на одном из заданных текстовых датасетов.

**Источник данных:** [Harry Potter and the Methods of Rationality](https://hpmor.ru/).

**Характер данных:** текст книги "Гарри Поттер и методы рационального мышления".

**Основные этапы:** исследовать следующие нейросетевые архитектуры:

1. Simple RNN с посимвольной и пословной токенизацией.
2. Однонаправленная однослойная и многослойная LSTM c посимвольной токенизацией и токенизацией по словам и [на основе BPE](https://keras.io/api/keras_nlp/tokenizers/byte_pair_tokenizer/).
3. Двунаправленная LSTM.
4. *(На хорошую оценку)* трансформерная архитектура (GPT) "с нуля" [пример](https://keras.io/examples/generative/text_generation_gpt/).
5. *(На отличную оценку)* дообучение предобученной GPT-сети [пример](https://github.com/ZotovaElena/RuGPT3_finetuning).

---

# Реализации

1. [RNN с посимвольной токенизацией](RNN_char.ipynb)
2. [RNN с пословной токенизацией](RNN_word.ipynb)
3. [Однонаправленная LSTM + BPE](LSTM_unidirectional_BPE.ipynb)
4. [Двунаправленная LSTM](LSTM_bidirectional.ipynb)
5. Архитектура GPT (текущий файл)
6. [Дообучение GPT](GPT_finetuning.ipynb)

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 1 Подготовка окружения

Импорт библиотек:

In [1]:
from typing import Tuple

import os

import tensorflow as tf

from tensorflow import keras
from tensorflow.data import TextLineDataset, AUTOTUNE
from keras import callbacks, utils, losses

import keras.layers as l

import keras_nlp

from keras_nlp import tokenizers, samplers, metrics

# custom funcs
import utils.web_scrapping as web
import utils.process_checking as check
import utils.data_preprocessing as data_prep
import utils.charts_plotting as chart

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 2 Загрузка данных

Формирование/загрузка набора данных в зависимости от его наличия:

In [2]:
data = web.load_data('https://hpmor.ru/', 'hpmor.txt')

Uploaded from data/hpmor.txt


Выведение на экран начала текста:

In [3]:
data[:500]

'гарри поттер и методы рационального мышления. элиезер юдковский (less wrong). петуния вышла замуж не за дурсля, а за университетского профессора, и гарри попал в гораздо более благоприятную среду. у него были частные учителя, дискуссии с отцом, а главное — книги, сотни и тысячи научных и фантастических книг. в 11 лет гарри знаком с квантовой механикой, когнитивной психологией, теорией вероятностей и другими вещами. но гарри не просто вундеркинд, у него есть загадочная тёмная сторона, которая явн'

Выведение на экран общего числа слов и предложений в тексте:

In [4]:
check.print_total(data)

Всего слов: 559791
Всего предложений: 37351


Задание путей до выборок данных:

In [5]:
path_train = 'data/hpmor_train.txt'
path_valid = 'data/hpmor_valid.txt'

Формирование тренировочной и валидационной выборок:

In [6]:
data_prep.train_valid_test_split_save(data, path_train, path_valid)

Files already exist


<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 3 Трансформерная архитектура GPT

### 3.1 Токенизация символов

Задание функции разделения данных на признаки и целевой признак:

In [7]:
def preprocess(inputs: TextLineDataset):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels

---

Выведение на экран максимальной и минимальной длины предложений в тексте:

In [8]:
check.print_max_min_len(data.split('.'))

Максимальная длина строки: 217
Минимальная длина строки: 0


Задание констант:

In [9]:
BATCH_SIZE = 64
SEQ_LEN = 128
MIN_SEQ_LEN = 450
MAX_VOCAB_LEN = 5000

Формирование датасетов:

In [10]:
train_dataset = (
    TextLineDataset(path_train)
    .filter(lambda x: tf.strings.length(x) > MIN_SEQ_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

valid_dataset = (
    TextLineDataset(path_valid)
    .filter(lambda x: tf.strings.length(x) > MIN_SEQ_LEN)
    .batch(BATCH_SIZE)
)

Выведение на экран первого бартча тренировочного датасета:

In [11]:
list(train_dataset.as_numpy_iterator())[0][0].decode('utf-8')[:500]

'профессор макгонагалл, у которой использование маховика времени не вошло в привычку настолько, как у директора и гарри, после окончания совещания сразу же ушла спать звук шёл секунд двадцать, поэтому, наверное, минуты две на метле… движением настолько гладким, что оно казалось неосознанным, профессор макгонагалл приняла нужную позу: — экспекто патронум сгусток красного света почти в упор ударил в затылок не-сьюзен и швырнул её на пол — с ним всё в порядке, ему лишь нужен день отдыха — ты должен '

Формирование словаря:

In [ ]:
vocabulary = tokenizers.compute_word_piece_vocabulary(
    train_dataset,
    vocabulary_size=MAX_VOCAB_LEN,
    lowercase=True,
    reserved_tokens=['[PAD]', '[UNK]', '[BOS]'],
)

Выведение на экран общего числа слов:

In [ ]:
print('Слов в словаре:', len(vocabulary))

Выведение на экран первых элементов словаря:

In [ ]:
vocabulary[:10]

Формирование словаря токенов:

In [ ]:
tokenizer = tokenizers.WordPieceTokenizer(
    vocabulary=vocabulary,
    sequence_length=SEQ_LEN,
    lowercase=True
)

задание объекта, формирующего токен начала строки:

In [ ]:
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id('[BOS]'),
)

Токенизация датасетов:

In [ ]:
train = train_dataset.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
valid = valid_dataset.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

Выведение на экран экземпляра признаков и целевого признака:

In [ ]:
check.print_single_element(
    train.get_single_element()[0],
    train.get_single_element()[1],
    tokenizer.id_to_token
)

---

### 3.2 Подготовка модели

Задание функции, конструирующей модель:

In [ ]:
def model_construction(vocab_size: int,
                       seq_len: int,
                       embed_dim: int,
                       n_layers: int,
                       n_heads: int,
                       inter_dim: int) -> keras.Model:
    
    features = l.Input(shape=(None,), dtype=tf.int32)
    
    embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
        vocabulary_size=vocab_size,
        sequence_length=seq_len,
        embedding_dim=embed_dim,
        mask_zero=True,
    )
    
    x = embedding_layer(features)
    
    for _ in range(n_layers):
        decoder_layer = keras_nlp.layers.TransformerDecoder(
            num_heads=n_heads,
            intermediate_dim=inter_dim,
        )
        x = decoder_layer(x)
        
    target = l.Dense(vocab_size)(x)
    
    model = keras.Model(inputs=features, outputs=target)
    
    return model

---

Задание модели:

In [ ]:
model = model_construction(
    vocab_size=MAX_VOCAB_LEN,
    seq_len=SEQ_LEN,
    embed_dim=256,
    n_layers=2,
    n_heads=3,
    inter_dim=256    
)

Выведение на экран таблицы поведения параметров на слоях нейросети:

In [ ]:
model.summary()

Проверка наличия папки для хранения изображений:

In [ ]:
if os.path.isdir('images/') == False:
    os.mkdir('images/')

Выведение на экран отображения послойной обработки данных моделью:

In [ ]:
utils.plot_model(model, 'images/gpt_arch.png', show_shapes=True, dpi=70)

---

### 3.3 Обучение модели

Проверка наличия папки для хранения контрольных точек:

In [ ]:
if os.path.isdir('checkpoints/') == False:
    os.mkdir('checkpoints/')

Задание пути для хранения контрольных точек:

In [ ]:
path_checkpoints = 'checkpoints/gpt_arch'

Проверка наличия папки для хранения контрольных точек:

In [ ]:
if os.path.isdir(path_checkpoints) == False:
    os.mkdir(path_checkpoints)

Задание коллбека точек сохранения:

In [ ]:
checkpoint_path = os.path.join(path_checkpoints, 'checkpoint_{epoch}')

checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    save_weights_only=True, 
    verbose=1
)

Подготовка модели к обучению:

In [ ]:
model.compile(
    optimizer='adam', 
    loss=losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

Обучение модели:

In [ ]:
history = model.fit(
    train, 
    validation_data=valid, 
    epochs=200, 
    verbose=2,
    callbacks=[checkpoint_callback]
)

Выведение на экран графика значений функции потерь и качества модели:

In [ ]:
chart.plot_loss_acc(history)

Сброс состояния модели:

In [ ]:
model.reset_states()

---

### 3.4 Генерация текста

Задание функции, заполняющей тело запроса:

In [ ]:
def next(prompt: tf.Tensor, cache: tf.Tensor, index: tf.Tensor) -> Tuple[tf.Tensor, None, Tuple]:
    logits = model(prompt)[:, index - 1, :]
    hidden_states = None
    return logits, hidden_states, cache

Задание функции предсказания следующего символа:

In [ ]:
def generate_text(sampler: keras_nlp.samplers) -> str:
    prompt = start_packer(tokenizer(['']))
    
    tokens = sampler(
        next=next,
        prompt=prompt,
        index=1,
    )
    
    text = tokenizer.detokenize(tokens)
    return text.numpy()[0].decode('utf-8')

---

**Greedy search** - алгоритм подбора наиболее вероятного токена на каждом шаге.

Генерация текста:

In [ ]:
generate_text(samplers.GreedySampler())

---

**Beam search** - алгоритм подбора наиболее вероятной последовательности на каждом шаге и лучшего токена среди всех последовательностей. 

Генерация текста:

In [ ]:
generate_text(samplers.BeamSampler(num_beams=10))

---

**Random search** - алгоритм подбора следующего наиболее вероятного токена, которым может оказаться любое слово из всего корпуса текста, на каждом шаге.

Генерация текста:

In [ ]:
generate_text(samplers.RandomSampler())

---

**Top-K search** - алгоритм подбора следующего токена по вероятностному распределению, которое распределеяется между фиксированного числа топ-К наиболее вероятных токенов.

Генерация текста:

In [ ]:
generate_text(samplers.TopKSampler(k=10))

---

**Top-P search** - алгоритм подбора следующего токена по вероятностному распределению, неравномерно распределённому среди наиболее вероятных токенов.

Генерация текста:

In [ ]:
generate_text(samplers.TopPSampler(p=0.5))

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 4 Общий вывод

Были продемонстрированы результаты обучения построенной модели с применением различных алгоритмов поиска лучших токенов.

<div style="text-align: center; font-size: 20px; padding: 15px 0;">
    <a href="#Содержание" data-toc-modified-id="Содержание" style="text-decoration: none; color: #296eaa; border: 2px dashed #296eaa; opacity: 0.8; border-radius: 3px; padding: 10px 80px;">
        В начало файла ↑
    </a>
</div>